<a href="https://colab.research.google.com/github/cu-applied-math/appm-4600-numerics/blob/main/Demos/Misc_CommonPythonPitfalls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Some common pitfalls in Python programming
Especially ones relevant to numerical analysis

Compiled by Stephen Becker, University of Colorado Boulder

In [1]:
import numpy as np

# Pass by reference vs Pass by value
If we give a function an input "x", are we giving it the value that "x" represents, or are we giving it the location in memory that "x" points to? If the latter, then if we update "x", we can have **side effects**

(Note: in **Matlab**, you have fewer issues about this, since it always makes a copy if the data is about to be changed. This is good in some ways, bad in other ways, since you have less control and can end up making unnecessary copies.  It uses a "lazy copy" mechanism, also known as "copy-on-write" (COW), see
https://www.mathworks.com/help/matlab/matlab_prog/avoid-unnecessary-copies-of-data.html )

In our first example, we don't have any side effects. Good!

In [6]:
def myFunction(x):
    for i in range(5):
        x = x + 1
    return x

x = 0
y = myFunction(x)
print(f'Input: {x}, Output: {y}')

x = np.array(x)
y = myFunction(x)
print(f'Input: {x}, Output: {y}')


Input: 0, Output: 5
Input: 0, Output: 5


But now let's make what seems like a very small change. Instead of `x=x+1`, let's do `x += 1`.  They accomplish the same thing, *except* that the former creates a **new copy** of the variable, and the latter **updates the variable in place**.

Confusingly, this only leads to side effects **sometimes**.  If we pass in `x=0`, then there are no side effects because Python treats this object as a value only. But if we pass in a numpy array, then Python doesn't pass in a copy of the value (for good reason... that would be slow if the array is large), but rather a datastructure that contains a pointer to the actual data. So now an **in place** operation has side effects:

In [20]:
def myFunction(x):
    for i in range(5):
        x+= 1
    return x


x = 0
y = myFunction(x)
print(f'Input: {x}, Output: {y}')

x = np.array(x)
y = myFunction(x)
print(f'Input: {x}, Output: {y}')

Input: 0, Output: 5
Input: 5, Output: 5


even weirder, if we update the value of the output afterwards, it affects the value of the input!  This is because y and x point to the same data

In [22]:
y[()] = 15
print(f'Input: {x}, Output: {y}')

Input: 15, Output: 15


We can prevent side effects by forcing an explicit copy:

In [23]:
def myFunction(x):
    if isinstance(x, np.ndarray):
        x = x.copy()
    for i in range(5):
        x+= 1
    return x


x = 0
y = myFunction(x)
print(f'Input: {x}, Output: {y}')

x = np.array(x)
y = myFunction(x)
print(f'Input: {x}, Output: {y}')

y[()] = 15
print(f'Input: {x}, Output: {y}')

Input: 0, Output: 5
Input: 0, Output: 5
Input: 0, Output: 15


### another example


In [24]:
def myFunction(x):
    if not isinstance(x, np.ndarray):
        raise ValueError('This function requires the input to be a numpy array')
    x = 10
    return x

x = np.array([0])
y = myFunction(x)
print(f'Input: {x}, Output: {y}')

Input: [0], Output: 10


In [25]:
def myFunction(x):
    if not isinstance(x, np.ndarray):
        raise ValueError('This function requires the input to be a numpy array')
    x[0] = 10
    return x

x = np.array([0])
y = myFunction(x)
print(f'Input: {x}, Output: {y}')

Input: [10], Output: [10]


Setting `x=10` implicitly forces a copy, while `x[0]=10`, while it seems like it's the same thing, just modifies the values and doesn't make a new copy

There's miscellaneous documentation for this on the web. A few sites:
- https://numpy.org/doc/stable/user/basics.copies.html on "Copies and views"

## JAX
By default, JAX is single precision (32 bit) rather than double (64 bit). For most scientific computing, double precision is standard, though for machine learning, 32 bit is more common (in part because GPUs have limited memory, so every bit counts)

To get JAX to use 64 bits, run the following code:

In [26]:
import jax
jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

x = jnp.ones(3)
print(x.dtype) # Now this is float64 not float32

float64


## Shapes
Unlike Matlab, Python has purely 1D arrays, i.e., size `(n,)`. Think of this as a vector in $\mathbb{R}^n$.

Programming wise, this is **not** the same as an array of size `(n,1)` (nor `(1,n)`).  Of course $\mathbb{R}^n$ and $\mathbb{R}^{n\times 1}$ are really the same, but for programming, you have to make the link explicit.

Things get complicated because Python tries to do stuff for you via **broadcasting**, and this often messes up math because Python is **row** based whereas most math treats a vector as a **column vector**.

In [27]:
x = np.arange(5)
print(x, x.shape)

[0 1 2 3 4] (5,)


In [28]:
print( x + 2 ) # this is broadcasting a vector with a scalar. Very convenient

[2 3 4 5 6]


In [29]:
print( x, x.T ) # what if we transpose a pure vector? Nothing happens!
print( x.shape, x.T.shape )

[0 1 2 3 4] [0 1 2 3 4]
(5,) (5,)


In [34]:
y = np.arange(5).reshape( (-1,1) ) # this is one way to make a column vector
y = np.arange(5)[:,None] # this is another way to make a column vector
print(y, y.shape )

[[0]
 [1]
 [2]
 [3]
 [4]] (5, 1)


In [35]:
print(x+y) # a vector + a vector is... a matrix?

[[0 1 2 3 4]
 [1 2 3 4 5]
 [2 3 4 5 6]
 [3 4 5 6 7]
 [4 5 6 7 8]]


What happened in the above? `x` is size `(5,)` and `y` is size `(5,1)`. These are not the same, so they cannot directly be added. So numpy tries to be helpful and **broadcasts** `x` into a 2D array, but it is row-based, so it broadcasts it into an array of size `(1,5)`.  Now, adding things of size `(5,1)` and `(1,5)` is also not OK, so numpy interprets this to mean that we want to duplicate/broadcast **both** to become `(5,5)`.

This is a bit more clear if we make `x` and `y` different lengths

In [36]:
x = np.arange(5)
y = np.arange(6).reshape( (-1,1) )
print(x+y)

[[0 1 2 3 4]
 [1 2 3 4 5]
 [2 3 4 5 6]
 [3 4 5 6 7]
 [4 5 6 7 8]
 [5 6 7 8 9]]


A common place this occurs is if `x` and `y` are the same (or nearly the same), but different shapes. We check to see if their difference is close to zero, and it appears that it is *not*!

In [37]:
x = np.arange(5)
y = np.arange(5).reshape( (-1,1) )

print(np.linalg.norm(x-y))

10.0


One fix, among others, is to always force a 2D array (of size `(n,1)` or `(1,n)`) into a pure vector of size `(n,)`. This can be done with the `ravel` or `flatten` methods, which are similar but slightly different.  From AI summary:

**ravel()**:
- Returns a **view** whenever possible: `ravel()` *attempts* to return a view of the original array, meaning it doesn't create a new array in memory if the underlying data can be represented as a 1D array without copying.
- Modifications affect the original array: If `ravel()` returns a view, any changes made to the raveled array will also be reflected in the original array.
- Faster and more memory-efficient: Because it often avoids creating a new copy, ravel() is generally faster and uses less memory than flatten().
Can be called on any object that can be successfully parsed: ravel() is a library-level function (`numpy.ravel()`) and can work with various array-like objects.

**flatten()**:
- Always returns a copy: `flatten()` always creates a new, independent copy of the array in memory.
- Modifications do not affect the original array: Changes made to the flattened array will not impact the original array because they are separate entities.
- Slower and uses more memory: Creating a new copy requires more time and memory compared to returning a view.
- Method of an ndarray object: flatten() is a method of NumPy arrays (`array.flatten()`) and can only be called on actual ndarray objects.

That is, this is related to the **side effects** discussion above

In [40]:
print(x.ravel(), np.ravel(x))  # equivalent. In this case, x was already size (n,), so ravel has no effect
print(x.shape, x.ravel().shape)

[0 1 2 3 4] [0 1 2 3 4]
(5,) (5,)


In [45]:
print(x.flatten())
# np.flatten(x) # you *cannot* do this. There is no np.flatten()
print(x.shape, x.flatten().shape)

[0 1 2 3 4]
(5,) (5,)


In [46]:
print(y.ravel())
print(y.shape, y.ravel().shape)

[0 1 2 3 4]
(5, 1) (5,)


In [47]:
print(y.flatten())
print(y.shape, y.flatten().shape)

[0 1 2 3 4]
(5, 1) (5,)


So, if we're looking for the norm of $x-y$, then we can do the following (using `ravel` rather than `flatten` since it's faster and we're not worried about side-effects in this case)

In [49]:
print(np.linalg.norm(x-y), np.linalg.norm(x.ravel()-y.ravel()) )

10.0 0.0
